In [2]:
import pandas as pd
import numpy as np

movie_path = r'C:\Users\user\OneDrive\桌面\course\DS\hw0\IMDB-Movie-Data.csv'
df = pd.read_csv(movie_path)

df['Actors'] = df['Actors'].apply(lambda x: [actor.strip() for actor in x.split('|')])
df['Revenue'] = df['Revenue (Millions)']
df['Runtime'] = df['Runtime (Minutes)']
df['Genre'] = df['Genre'].apply(lambda x: [gerne.strip() for gerne in x.split('|')])

# create an dataframe for actor
rows = []
df.apply(lambda x: [rows.append([actor, x.Title, x.Genre, x.Director, x.Year, x.Rating, x.Revenue, x.Actors]) for actor in x.Actors], axis=1)
df_act =pd.DataFrame(rows, columns=['Actor', 'Title', 'Genre', 'Director', 'Year', 'Rating', 'Revenue', 'Actors'])

In [3]:
# Top‐3 movies with the highest ratings in 2016?
df[df['Year'] == 2016].nlargest(3, 'Rating')

,Rank,Title,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Revenue,Runtime
2,118,Dangal,"[Action, Biography, Drama]",Nitesh Tiwari,"[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh...",2016,161,8.8,48969,11.15,NaN,11.15,161
4,97,Kimi no na wa,"[Animation, Drama, Fantasy]",Makoto Shinkai,"[Ryunosuke Kamiki, Mone Kamishiraishi, Ryo Nar...",2016,106,8.6,34110,4.68,79.0,4.68,106
15,862,Koe no katachi,"[Animation, Drama, Romance]",Naoko Yamada,"[Miyu Irino, Saori Hayami, Aoi Yuki, Kensho Ono]",2016,129,8.4,2421,NaN,80.0,NaN,129


In [4]:
# The actor generating the highest average revenue?

test = df_act.groupby('Actor').mean()
pd.DataFrame(test.loc[:, 'Revenue'].nlargest(1, keep='all'))

,Revenue
Actor,
Daisy Ridley,936.63
John Boyega,936.63


In [5]:
# The average rating of Emma Watson’s movies?

test = df_act.groupby('Actor').mean()
test.loc['Emma Watson', 'Rating']

7.175

In [6]:
# Top‐3 directors who collaborate with the most actors? 
pd.DataFrame(df[['Director', 'Actors']].groupby('Director').sum().apply(lambda x: len(set(x[0])), axis=1).nlargest(3, keep='all'))

,0
Director,
Ridley Scott,28
M. Night Shyamalan,24
Danny Boyle,20
Paul W.S. Anderson,20


In [7]:
# Top‐2 actors playing in the most genres of movies? 
pd.DataFrame(df_act[['Actor', 'Genre']].groupby('Actor').sum().apply(lambda x: len(set(x[0])), axis=1).nlargest(2, keep='all'))

,0
Actor,
Brad Pitt,14
Amy Adams,13
Chloe Grace Moretz,13
Hugh Jackman,13
Johnny Depp,13
Scarlett Johansson,13


In [8]:
# Top‐3 actors whose movies lead to the largest maximum gap of years?
temp = df_act[['Actor', 'Year']].groupby('Actor')
(temp.max()-temp.min()).nlargest(1, columns='Year', keep='all')

,Year
Actor,
Abbie Cornish,10
Anne Hathaway,10
Audrey Tautou,10
Ben Kingsley,10
Ben Whishaw,10
Bob Balaban,10
Brad Pitt,10
Bryce Dallas Howard,10
Chiwetel Ejiofor,10


In [27]:
# Find all actors who collaborate with Johnny Depp in direct and indirect ways

from collections import deque

relations = dict(zip(df_act['Actor'].unique(), [set() for i in range(len(df_act['Actor'].unique()))]))

for i, row in df.iterrows():
    for actor1 in row['Actors']:
        for actor2 in row['Actors']:
            if actor1 != actor2:
                relations[actor1].add(actor2)

# create an dictionary which actors are keys and directly related actors are value
for actor in relations:
    relations[actor] = list(relations[actor])

# create an queue save the directly related actor for Johnny Depp
stack = deque(relations['Johnny Depp'])
Johnny_friends = []

# pop  friend in stack and add it to the result
# push all friend of the poped friend into the stack if they are not in the result
# run until the stack is empty
while stack:

    old_friend = stack.popleft()
    for new_friend in relations[old_friend]:
        if not new_friend in stack and not new_friend in Johnny_friends:
            stack.append(new_friend)
    Johnny_friends.append(old_friend)

Johnny_friends = [Johnny_friend for Johnny_friend in Johnny_friends if Johnny_friend != 'Johnny Depp']
print(len(Johnny_friends))
pd.DataFrame(data=Johnny_friends, columns=['friend'])

1574
